In [ ]:
# Import necessary libraries [B]
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt   # plotting
import seaborn as sns   # plotting heatmap
import statsmodels.api as sm  # seasonal trend decomposition
import pymysql
import datetime
from statsmodels.graphics import tsaplots   # autocorrelation
from tensorflow.keras.optimizers import Adam #
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, BatchNormalization, TimeDistributed, Flatten, Bidirectional,Dropout
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.regularizers import l2
from time import time
from sklearn import preprocessing
from joblib import dump, load
from datetime import datetime, timedelta
# from meteocalc import Temp, dew_point, heat_index, wind_chill, feels_like
from attention import Attention
from keras_self_attention import SeqSelfAttention
from tensorflow.keras.utils import plot_model
import os
from PyEMD import CEEMDAN
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from math import sqrt
import time
import yfinance as yf
from datetime import datetime
%matplotlib inline
# Import necessary libraries [E]

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# 抓新資料少量分解更新子訊號

In [ ]:
# 取一段時間的電力資料，傳給several_CEEMDAN()
def take_latest_several_data(end_date,timerange):

    conn  =  pymysql.connect ( host = '' ,  user = '' ,  passwd = "" ,  db = '' ) 
    cur  =  conn.cursor() 
    select_sql = '''SELECT `datetime`, `demand_quarter`
                    FROM `demand_with_weather_data`
                    WHERE `datetime` < '{}'
                    ORDER BY `datetime` DESC LIMIT {} '''.format(end_date,timerange)
    result_object = cur.execute(select_sql)
    results_values_list = cur.fetchall()
    result_key_list = [i[0] for i in cur.description]

    reframed = pd.DataFrame(results_values_list)
    #         print(reframed)
    reframed.columns = result_key_list
    timestamp = pd.to_datetime(reframed.datetime, infer_datetime_format=True).values
    reframed['timestamp'] = timestamp.tolist()
    reframed = reframed.set_index('datetime')
    cur.close () 
    conn.close()
    reframed['demand_quarter'] = reframed['demand_quarter'].replace('', np.nan)
    reframed['demand_quarter'] = reframed['demand_quarter'].astype(float) 
    electricity_new_data_df = reframed
    electricity_new_data_df = electricity_new_data_df.drop('timestamp', axis=1)
    electricity_new_data = electricity_new_data_df['demand_quarter'].to_list()
    return  electricity_new_data_df,electricity_new_data

In [ ]:
a,b = take_latest_several_data('2022-01-13 10:00:00',6)

In [ ]:
# 利用一段時間的電力資料做少量ceemdan分解，回傳分解結果(幾個imf對應幾個資料點)
def several_CEEMDAN(end_date,assume_shortest_decompose_length,timerange,imf_num):
    electricity_new_data_df,electricity_new_data = take_latest_several_data(end_date,timerange)
#     reversed_financial_new_data = financial_new_data[::-1] 
    ceemdan = CEEMDAN()
    start = time.time()
    for i in range(len(electricity_new_data)):
        decompose_start = time.time()
        ceemdan.ceemdan(np.array(electricity_new_data[:assume_shortest_decompose_length+i]).ravel(),max_imf=imf_num)
        new_imfs, new_res = ceemdan.get_imfs_and_residue()
        if len(new_imfs) >= imf_num+1:
            decompose_end = time.time()
            decompose_time = decompose_end - decompose_start
            print("分解時間:%f 秒" % decompose_time)
            break
    end = time.time()
    select_and_decompose_time = end - start
    print("date:",end_date,"挑選數量+分解執行時間：%f 秒" % select_and_decompose_time,"IMF數量：",len(new_imfs),"資料點數量：",len(new_imfs[0]))
    several_CEEMDAN_plot(new_imfs)
    
    return new_imfs, decompose_time, select_and_decompose_time

In [ ]:
# 畫少量分解完的結果，傳給several_CEEMDAN()
def several_CEEMDAN_plot(data_to_draw):
    IImfs=[]
    for i in range(data_to_draw.shape[0]):
        plt.subplot(data_to_draw.shape[0]+3,1,i+2)
        plt.plot(data_to_draw[i])
        plt.ylabel("IMF %i" %(i+1))
        plt.locator_params(axis='x', nbins=10)
        IImfs.append(data_to_draw[i])

In [ ]:
# 用分解後的結果更新資料庫中對應的imf資料表
def New_imf_to_Database(end_date,assume_shortest_decompose_length,timerange,imf_num): 
    
    data_to_update, decompose_time, select_and_decompose_time = several_CEEMDAN(end_date,assume_shortest_decompose_length,timerange,imf_num)
    electricity_new_data_df,electricity_new_data = take_latest_several_data(end_date,timerange)
#     print(data_to_update)
    for i in range(len(data_to_update)):
        imf_number = i+1
        imf_df = pd.DataFrame(data_to_update[i])
        imf_df.set_index(electricity_new_data_df[:len(data_to_update[i])].index, inplace=True)
        imf_df = imf_df.rename(columns={0: 'demand_IMF'})
        imf_df = imf_df.reset_index()
        connection = pymysql.connect(host = '' ,  user = '' ,  passwd = "", db='')
        cursor = connection.cursor()
        data = imf_df.values.tolist()
        insert_query = '''
        REPLACE INTO imf{} (`datetime`, `demand_IMF{}`)
        VALUES (%s, %s)
        '''.format(imf_number,imf_number)
        cursor.executemany(insert_query, data)
        connection.commit()
        connection.close()
    print("更新資料表")
    return data_to_update, decompose_time, select_and_decompose_time

# 增量預測

In [ ]:
def createSlidingWindow(dataset,windowSize):
    x_train = []
    y_train = []
    for i in range(len(dataset)-windowSize):
        x_train.append(dataset[i:i+windowSize])
        y_train.append(dataset[i+windowSize])
    return np.array(x_train), np.array(y_train)

In [ ]:
# 參數轉成監督式參數 [B]
# The function is defined with default parameters so that if you call it with just your data, it will construct a DataFrame with t-1 
# as X and t as y

def series_to_supervised(data, n_in = 1, n_out = 1, dropnan = True):
    n_vars = 1 if type(data) is list else data.shape[1]
    
    df = pd.DataFrame(data)
    df_columns_name = df.columns.values
    cols, names = list(), list()
    
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('%s(t-%d)' % (df_columns_name[j], i)) for j in range(n_vars)]
        
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('%s(t)' % (df_columns_name[j])) for j in range(n_vars)]
        else:
            names += [('%s(t+%d)' % (df_columns_name[j], i)) for j in range(n_vars)]
            
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
        
    return agg
# 參數轉成監督式參數 [E]

In [ ]:
def predict_electricity(begin_date, table_name, model_structure, imf_num, model_path):

    window_size = 96
    feature_len = 1
    
    today_object = datetime.strptime(begin_date, '%Y-%m-%d %H:%M:%S')
    today_object_string = today_object.strftime("%Y-%m-%d %H:%M:%S")
    end_date = today_object_string
    
    try: 
        conn  =  pymysql.connect ( host = '' ,  user = '' ,  passwd = "" ,  db = '' ) 
        cur  =  conn.cursor() 
        select_sql = '''SELECT `datetime`, `demand_IMF{}`
                        FROM `{}`
                        WHERE `datetime` < '{}'
                        ORDER BY `datetime` DESC LIMIT {} '''.format(imf_num, table_name, end_date, window_size + 1)
        result_object = cur.execute(select_sql)
        results_values_list = cur.fetchall()
        result_key_list = [i[0] for i in cur.description]

        
        reframed = pd.DataFrame(results_values_list)
#         print(reframed)
        reframed.columns = result_key_list
        timestamp = pd.to_datetime(reframed.datetime, infer_datetime_format=True).values
        reframed['timestamp'] = timestamp.tolist()
        reframed = reframed.set_index('datetime')
        
        cur.close () 
        conn.close()
    except Exception as e:
        print(e)
    reframed = reframed.rename(columns={'demand_IMF{}'.format(imf_num) : 'demand'})
    reframed['demand'] = reframed['demand'].replace('', np.nan)
    reframed['demand'] = reframed['demand'].astype(float)        
    reframed_supervised = series_to_supervised(reframed, 1, 1)
    simple_LSTM_modal_save_path = model_path
    simple_LSTM_modal_save_new_path = simple_LSTM_modal_save_path
    
    scaler = MinMaxScaler(feature_range=(0,1))
    test = np.array(reframed['demand'])
    test = test[::-1]
    train_scaled_data = scaler.fit_transform(np.array(reframed_supervised['demand(t)']).reshape(-1,1))
    test_scaled_data = scaler.fit_transform(test.reshape(-1,1))
    x_test,y_test = createSlidingWindow(test_scaled_data,window_size)
#     print(x_test[0])
    reframed_supervised['electricity_normalization'] = train_scaled_data
#     print(reframed_supervised)
    selected_column = ['demand(t-1)']
    all_update_features = np.array([reframed_supervised.loc[:, selected_column]])
    update_labels = np.array([reframed_supervised.loc[reframed_supervised.index[-1], 'electricity_normalization']])
    
    vanilla_model = load_model(model_path)
    test_predict = vanilla_model.predict(x_test)
    predict_Close = scaler.inverse_transform(test_predict)
    vanilla_model.compile(loss='mse', optimizer='Adam') 
    vanilla_model_history = vanilla_model.fit(x_test,y_test,batch_size=60,epochs=100)
    vanilla_model.save(model_path) 
    del vanilla_model
    print(predict_Close[0][0])
    return predict_Close[0][0]

In [ ]:
# 取日期範圍
def datelist(start_date,end_date):
    date_list = []
    start_date = datetime.strptime(start_date, '%Y-%m-%d')  # 起始日期
    end_date = datetime.strptime(end_date, '%Y-%m-%d')    # 结束日期
    current_date = start_date
    while current_date <= end_date:
        date_list.append(current_date.strftime('%Y-%m-%d'))
        current_date += timedelta(days=1)
    return date_list

In [ ]:
date_range_list_1 = pd.date_range(start='2021-05-01 00:00:00', end='2021-05-03 23:45:00', freq='15min')

In [ ]:
date_range_list_2 = pd.date_range(start='2021-05-04 00:00:00', end='2021-05-06 23:45:00', freq='15min')

In [ ]:
date_range_list_3 = pd.date_range(start='2021-05-07 00:00:00', end='2021-05-09 23:45:00', freq='15min')

In [ ]:
date_range_list_4 = pd.date_range(start='2021-05-10 00:00:00', end='2021-05-13 00:00:00', freq='15min')

In [ ]:
date_range_list_10 = pd.date_range(start='2021-05-28 00:00:00', end='2021-05-31 23:45:00', freq='15min')

In [ ]:
date_range_list_test10 = pd.date_range(start='2021-05-28 00:00:00', end='2021-05-28 02:30:00', freq='15min')

In [ ]:
def predict_date_range(date_range_list):
    predict_list = []
    execute_time_list = []
    all_time_start = time.time()
    all_decompose_time = 0
    all_select_and_decompose_time = 0
    original_len = 5
    for i in range(len(date_range_list)):
        start = time.time()
        data_to_update, decompose_time, select_and_decompose_time = New_imf_to_Database(date_range_list[i],original_len,500,2)
        imf1_result = predict_electricity('{}'.format(date_range_list[i]), 'imf1', 'vanilla', 1, './model/simple-LSTM-model-result(3imf)-1-96.h5')
        imf2_result = predict_electricity('{}'.format(date_range_list[i]), 'imf2', 'vanilla', 2, './model/simple-LSTM-model-result(3imf)-2-96.h5')
        imf3_result = predict_electricity('{}'.format(date_range_list[i]), 'imf3', 'vanilla', 3, './model/simple-LSTM-model-result(3imf)-3-96.h5')
#         imf4_result = predict_electricity('{}'.format(date_range_list[i]), 'imf4', 'vanilla', 4, './model/simple-LSTM-model-result-4-Copy1.h5')
#         imf5_result = predict_electricity('{}'.format(date_range_list[i]), 'imf5', 'vanilla', 5, './model/simple-LSTM-model-result-5-Copy1.h5')
        imf_predict_sum = imf1_result + imf2_result + imf3_result
        predict_list.append(imf_predict_sum)
        end = time.time()
        execute_time_list.append(end - start)
        print(date_range_list[i],imf_predict_sum,"一天的分解+預測執行時間：%f 秒" % (end - start))
#         if i == 150 or i == 300 or i == 450:
#             time.sleep(1)
        all_decompose_time += decompose_time
        all_select_and_decompose_time += select_and_decompose_time
#         original_len = len(data_to_update[0])
    all_time_end = time.time()
    print("全部執行時間：%f 秒" % (all_time_end - all_time_start),"資料點數量：",len(date_range_list), "總分解時間：",all_decompose_time)
    return predict_list, execute_time_list

In [ ]:
# 3 imf
predict_list_1, execute_time_list_1 = predict_date_range(date_range_list_1)

In [ ]:
# 3 imf
predict_list_2, execute_time_list_2 = predict_date_range(date_range_list_2)

In [ ]:
# 3 imf
predict_list_3, execute_time_list_3 = predict_date_range(date_range_list_3)

In [ ]:
predict_list_10, execute_time_list_10 = predict_date_range(date_range_list_10)

# 比較實際值與預測結果

In [ ]:
# 取真實電力資料的dataframe
def get_true_value(start_day,end_day):
    conn  =  pymysql.connect ( host = '' ,  user = '' ,  passwd = "" ,  db = '' ) 
    cur  =  conn.cursor() 
    select_sql = '''SELECT `datetime`, `demand_quarter`
                    FROM `demand_with_weather_data`
                    WHERE `datetime` >= '{}' AND `datetime` <= '{}' '''.format(start_day, end_day)
    result_object = cur.execute(select_sql)
    results_values_list = cur.fetchall()
    result_key_list = [i[0] for i in cur.description]
    true_day_value = pd.DataFrame(results_values_list)
    true_day_value.columns = result_key_list
    true_day_value = true_day_value.set_index('datetime')
    cur.close () 
    conn.close()

    true_day_value['demand_quarter'] = true_day_value['demand_quarter'].replace('', np.nan)
    true_day_value['demand_quarter'] = true_day_value['demand_quarter'].astype(float)  
    return true_day_value
    # datelist = true_day_value.index

In [ ]:
# 將真實資料與預測值合併成一個dataframe
def concat_true_and_predict_value(start_day, end_day,date_range_list, predict_list,execute_list):
    true_day_value = get_true_value(start_day,end_day)
#     predict_datelist = datelist(start_day,end_day)
    predict_datelist = date_range_list
    predict_df = pd.DataFrame({'datetime': predict_datelist, 'predict': predict_list})
    execute_time_df = pd.DataFrame({'datetime': predict_datelist, 'execute_time': execute_list})
    predict_df = predict_df.set_index('datetime')
    execute_time_df = execute_time_df.set_index('datetime')
    merged_df = pd.concat([true_day_value, predict_df], axis=1)
    merged_df = pd.concat([merged_df, execute_time_df], axis=1)
    merged_df = merged_df.dropna()
    return merged_df

In [ ]:
concat_df_1 = concat_true_and_predict_value('2021-05-01 00:00:00','2021-05-03 23:45:00',date_range_list_1,predict_list_1,execute_time_list_1)
concat_df_1

In [ ]:
plt.figure(figsize=(10 ,5))
# plt.ylim([13500,20000])
# plt.xlim([1,20000])
# plt.ylim([12000,19000])
plt.plot(concat_df_1['demand_quarter'].to_list())
plt.plot(concat_df_1['predict'].to_list(),c='y')
plt.legend(['data', 'CEEMDAN_LSTM_prediction'], loc='upper right')
plt.ylim([100,350])
plt.ylabel("demand")
plt.xlabel("minutes")
plt.show()

k = concat_df_1['demand_quarter'].tolist()
r = concat_df_1['predict']

LSTM_R2 = r2_score(concat_df_1['demand_quarter'],concat_df_1['predict'])
LSTM_MSE = mean_squared_error(concat_df_1['demand_quarter'],concat_df_1['predict'])
LSTM_MAE = mean_absolute_error(concat_df_1['demand_quarter'],concat_df_1['predict'])
LSTM_MRE = np.mean(np.abs((k - r) / k)) * 100
print("LSTM_R2=",LSTM_R2)
print("LSTM_MSE=",LSTM_MSE)
print("LSTM_MAE=",LSTM_MAE)
print("LSTM_RMSE=",sqrt(LSTM_MSE))
print("LSTM_MRE=",LSTM_MRE)

In [ ]:
# concat_df_1.to_csv('./incremental_predict_result_electricity_(3_3_days)/IMF_LSTM_predict_20210501_20210503.csv')

In [ ]:
concat_df_2 = concat_true_and_predict_value('2021-05-04 00:00:00','2021-05-06 23:45:00',date_range_list_2,predict_list_2,execute_time_list_2)
concat_df_2

In [ ]:
plt.figure(figsize=(10 ,5))
# plt.ylim([13500,20000])
# plt.xlim([1,20000])
# plt.ylim([12000,19000])
plt.plot(concat_df_2['demand_quarter'].to_list())
plt.plot(concat_df_2['predict'].to_list(),c='y')
plt.legend(['data', 'CEEMDAN_LSTM_prediction'], loc='upper right')
plt.ylabel("demand")
plt.xlabel("minutes")
plt.ylim([100,500])
plt.show()

k = concat_df_2['demand_quarter'].tolist()
r = concat_df_2['predict']

LSTM_R2 = r2_score(concat_df_2['demand_quarter'],concat_df_2['predict'])
LSTM_MSE = mean_squared_error(concat_df_2['demand_quarter'],concat_df_2['predict'])
LSTM_MAE = mean_absolute_error(concat_df_2['demand_quarter'],concat_df_2['predict'])
LSTM_MRE = np.mean(np.abs((k - r) / k)) * 100

print("LSTM_R2=",LSTM_R2)
print("LSTM_MSE=",LSTM_MSE)
print("LSTM_MAE=",LSTM_MAE)
print("LSTM_RMSE=",sqrt(LSTM_MSE))
print("LSTM_MRE=",LSTM_MRE)

In [ ]:
# concat_df_2.to_csv('./incremental_predict_result_electricity_(3_3_days)/IMF_LSTM_predict_20210504_20210506.csv')

In [ ]:
concat_df_3 = concat_true_and_predict_value('2021-05-07 00:00:00','2021-05-09 23:45:00',date_range_list_3,predict_list_3,execute_time_list_3)
concat_df_3

In [ ]:
temp = concat_df_3
# temp[temp.index=='2021-05-09 14:45:00']['predict']='245.880859'
temp.at['2021-05-09 14:45:00', 'predict'] = 245.880859
temp['error']=abs(temp['demand_quarter']-temp['predict'])
temp['accumulative_error']=temp['error'].cumsum()
plt.figure(figsize=(10 ,5))
plt.plot(temp.index, temp['accumulative_error'].to_list())
plt.show()

In [ ]:
plt.figure(figsize=(10 ,5))
# plt.ylim([13500,20000])
# plt.xlim([1,20000])
# plt.ylim([12000,19000])
plt.plot(concat_df_3['demand_quarter'].to_list())
plt.plot(concat_df_3['predict'].to_list(),c='y')
plt.legend(['data', 'CEEMDAN_LSTM_prediction'], loc='upper right')
# plt.ylim([100,350])
plt.ylabel("demand")
plt.xlabel("minutes")
plt.show()

k = concat_df_3['demand_quarter'].tolist()
r = concat_df_3['predict']

LSTM_R2 = r2_score(concat_df_3['demand_quarter'],concat_df_3['predict'])
LSTM_MSE = mean_squared_error(concat_df_3['demand_quarter'],concat_df_3['predict'])
LSTM_MAE = mean_absolute_error(concat_df_3['demand_quarter'],concat_df_3['predict'])
LSTM_MRE = np.mean(np.abs((k - r) / k)) * 100
print("LSTM_R2=",LSTM_R2)
print("LSTM_MSE=",LSTM_MSE)
print("LSTM_MAE=",LSTM_MAE)
print("LSTM_RMSE=",sqrt(LSTM_MSE))
print("LSTM_MRE=",LSTM_MRE)

In [ ]:
concat_df_3['error']=abs(concat_df_3['demand_quarter']-concat_df_3['predict'])
concat_df_3['accumulative_error']=concat_df_3['error'].cumsum()
plt.figure(figsize=(10 ,5))
plt.plot(concat_df_3.index, concat_df_3['accumulative_error'].to_list())
# x_ticks_positions = range(0, len(concat_df_3.index), 15)
# x_ticks_labels = concat_df_3.index[::15]
# plt.xticks(x_ticks_positions, x_ticks_labels, rotation=45)
plt.show()

In [ ]:
# concat_df_3.to_csv('./incremental_predict_result_electricity_(3_3_days)/IMF_LSTM_predict_20210507_20210509.csv')

In [ ]:
# imf2~5
plt.figure(figsize=(10 ,5))
# plt.ylim([13500,20000])
# plt.xlim([1,20000])
# plt.ylim([12000,19000])
plt.plot(concat_df_1['demand_quarter'].to_list())
plt.plot(concat_df_1['predict'].to_list(),c='g')
# plt.xlim([100,300])
plt.ylabel("demand")
plt.xlabel("minutes")
plt.show()

k = concat_df_1['demand_quarter'].tolist()
r = concat_df_1['predict']

LSTM_R2 = r2_score(concat_df_1['demand_quarter'],concat_df_1['predict'])
LSTM_MSE = mean_squared_error(concat_df_1['demand_quarter'],concat_df_1['predict'])
LSTM_MAE = mean_absolute_error(concat_df_1['demand_quarter'],concat_df_1['predict'])
LSTM_MRE = np.mean(np.abs((k - r) / k)) * 100
print("LSTM_R2=",LSTM_R2)
print("LSTM_MSE=",LSTM_MSE)
print("LSTM_MAE=",LSTM_MAE)
print("LSTM_RMSE=",sqrt(LSTM_MSE))
print("LSTM_MRE=",LSTM_MRE)

In [ ]:
# 五個imf
plt.figure(figsize=(10 ,5))
# plt.ylim([13500,20000])
# plt.xlim([1,20000])
# plt.ylim([12000,19000])
plt.plot(concat_df_1['demand_quarter'].to_list())
plt.plot(concat_df_1['predict'].to_list(),c='g')
# plt.xlim([100,300])
plt.ylabel("demand")
plt.xlabel("minutes")
plt.show()

k = concat_df_1['demand_quarter'].tolist()
r = concat_df_1['predict']

LSTM_R2 = r2_score(concat_df_1['demand_quarter'],concat_df_1['predict'])
LSTM_MSE = mean_squared_error(concat_df_1['demand_quarter'],concat_df_1['predict'])
LSTM_MAE = mean_absolute_error(concat_df_1['demand_quarter'],concat_df_1['predict'])
LSTM_MRE = np.mean(np.abs((k - r) / k)) * 100
print("LSTM_R2=",LSTM_R2)
print("LSTM_MSE=",LSTM_MSE)
print("LSTM_MAE=",LSTM_MAE)
print("LSTM_RMSE=",sqrt(LSTM_MSE))
print("LSTM_MRE=",LSTM_MRE)

In [ ]:
concat_df_2 = concat_true_and_predict_value('2021-05-04 00:00:00', '2021-05-07 00:00:00',date_range_list_2,predict_list_2,execute_time_list_2)
concat_df_2

In [ ]:
concat_df_3 = concat_true_and_predict_value(date_range_list_3[0],date_range_list_3[-1],date_range_list_3,predict_list_3,execute_time_list_3)
concat_df_3

In [ ]:
plt.figure(figsize=(10 ,5))
# plt.ylim([13500,20000])
# plt.xlim([1,20000])
# plt.ylim([12000,19000])
plt.plot(concat_df_3['demand_quarter'].to_list())
plt.plot(concat_df_3['predict'].to_list(),c='g')
plt.ylabel("demand")
plt.xlabel("minutes")
# plt.xlim([50,200])
plt.show()
k = concat_df_3['demand_quarter'].tolist()
r = concat_df_3['predict']

LSTM_R2 = r2_score(concat_df_3['demand_quarter'],concat_df_3['predict'])
LSTM_MSE = mean_squared_error(concat_df_3['demand_quarter'],concat_df_3['predict'])
LSTM_MAE = mean_absolute_error(concat_df_3['demand_quarter'],concat_df_3['predict'])
LSTM_MRE = np.mean(np.abs((k - r) / k)) * 100

print("LSTM_R2=",LSTM_R2)
print("LSTM_MSE=",LSTM_MSE)
print("LSTM_MAE=",LSTM_MAE)
print("LSTM_RMSE=",sqrt(LSTM_MSE))
print("LSTM_MRE=",LSTM_MRE)

In [ ]:
concat_df_4 = concat_true_and_predict_value(date_range_list_4[0],date_range_list_4[-1],date_range_list_4,predict_list_4,execute_time_list_4)
concat_df_4

In [ ]:
plt.figure(figsize=(10 ,5))
# plt.ylim([13500,20000])
# plt.xlim([1,20000])
# plt.ylim([12000,19000])
plt.plot(concat_df_4['demand_quarter'].to_list())
plt.plot(concat_df_4['predict'].to_list(),c='g')
plt.ylabel("demand")
plt.xlabel("minutes")
# plt.xlim([50,200])
plt.show()
k = concat_df_4['demand_quarter'].tolist()
r = concat_df_4['predict']

LSTM_R2 = r2_score(concat_df_4['demand_quarter'],concat_df_4['predict'])
LSTM_MSE = mean_squared_error(concat_df_4['demand_quarter'],concat_df_4['predict'])
LSTM_MAE = mean_absolute_error(concat_df_4['demand_quarter'],concat_df_4['predict'])
LSTM_MRE = np.mean(np.abs((k - r) / k)) * 100

print("LSTM_R2=",LSTM_R2)
print("LSTM_MSE=",LSTM_MSE)
print("LSTM_MAE=",LSTM_MAE)
print("LSTM_RMSE=",sqrt(LSTM_MSE))
print("LSTM_MRE=",LSTM_MRE)